In [1]:
%%javascript
function toggler(){
    if(window.already_toggling){
        // Don't add multiple buttons.
        return 0
    }
    let btn = $('.input').append('<button>Toggle Code</button>')
        .children('button');
    btn.on('click', function(e){
        let tgt = e.currentTarget;
        $(tgt).parent().children('.inner_cell').toggle()
    })
    window.already_toggling = true;
}
// Since javascript cells are executed as soon as we load
// the notebook (if it's trusted), and this cell might be at the
// top of the notebook (so is executed first), we need to
// allow time for all of the other code cells to load before
// running. Let's give it 5 seconds.

setTimeout(toggler, 5000);

<IPython.core.display.Javascript object>

In [2]:
import ipywidgets as widgets
import sys
from pathlib import Path
import os

In [3]:
module_path='preprocessing/day_intervals_preproc'
if module_path not in sys.path:
    sys.path.append(module_path)
root_dir = os.path.dirname(os.path.abspath('UserInterface.ipynb'))
from day_intervals_cohort import *

# Welcome to your MIMIC-IV Project

A. Extract Data

1. Please select below if you want to work with ICU or Non-ICU data ?

In [4]:
radio_input1 = widgets.RadioButtons(options=['ICU', 'Non-ICU'],value='ICU')
radio_input1

RadioButtons(options=('ICU', 'Non-ICU'), value='ICU')

2. Please select what prediction task you want to perform ?

In [5]:
radio_input2 = widgets.RadioButtons(options=['30 Day Readmission','60 Day Readmission','90 Day Readmission','120 Day Readmission', 'Mortality'],value='30 Day Readmission')
radio_input2

RadioButtons(options=('30 Day Readmission', '60 Day Readmission', '90 Day Readmission', '120 Day Readmission',…

3. Please select if you want to perform choosen prediction task for a specific disease.

In [6]:
radio_input3 = widgets.RadioButtons(options=['No Disease Filter','Heart Failure','CKD','Respiratory Failure','COPD'],value='No Disease Filter')
radio_input3

RadioButtons(options=('No Disease Filter', 'Heart Failure', 'CKD', 'Respiratory Failure', 'COPD'), value='No D…

In [7]:
extract_button = widgets.Button(description='Extract Data')
extract_button

Button(description='Extract Data', style=ButtonStyle())

In [8]:
summary = extract_data(radio_input1.value,radio_input2.value, root_dir)

EXTRACTING FOR: | ICU | 30 DAY READMISSION |


TypeError: 'module' object is not callable

Below is summary of cohort selected according to your choices -

B. Lets Clean data and get it ready for model learning!

1. Do you want to group ICD 10 DIAG codes ?

In [8]:
radio_input4 = widgets.RadioButtons(options=['Yes','No'],value='Yes')
radio_input4

RadioButtons(options=('Yes', 'No'), value='Yes')

2. Do you want to group Medication codes to use Non propietary?

In [9]:
radio_input5 = widgets.RadioButtons(options=['Yes','No'],value='Yes')
radio_input5

RadioButtons(options=('Yes', 'No'), value='Yes')

3. Do you want to remove outliers for all features?

In [10]:
radio_input6 = widgets.RadioButtons(options=['Yes','No'],value='Yes')
radio_input6

RadioButtons(options=('Yes', 'No'), value='Yes')

4. What time bucket size you want to choose ?

In [11]:
radio_input7 = widgets.RadioButtons(options=['1 hour','2 hour','3 hour','4 hour','5 hour'],value='1 hour')
radio_input7

RadioButtons(options=('1 hour', '2 hour', '3 hour', '4 hour', '5 hour'), value='1 hour')

5. Do you want data for whole admission or ?

In [12]:
radio_input8 = widgets.RadioButtons(options=['All','Last 24 hours','Last 48 hours','Last 72 hours'],value='All')
radio_input8

RadioButtons(options=('All', 'Last 24 hours', 'Last 48 hours', 'Last 72 hours'), value='All')

In [13]:
clean_button = widgets.Button(description='Preprocess Data')
clean_button

Button(description='Preprocess Data', style=ButtonStyle())

D. Evaluation